In [ ]:
#import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel

In [2]:
### main wikidata df
wikidata = pd.read_html('assets/wikipedia_data.html')[1]
wikidata = wikidata[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'MLB', 'NBA', 'NHL']][0:51]

#clean wikidata df
wikidata['Population (2016 est.)[8]'].apply(int)
wikidata['NFL'] = wikidata['NFL'].str.replace(r"\[.*\]","")
wikidata['MLB'] = wikidata['MLB'].str.replace(r"\[.*\]","")
wikidata['NBA'] = wikidata['NBA'].str.replace(r"\[.*\]","")
wikidata['NHL'] = wikidata['NHL'].str.replace(r"\[.*\]","")

In [3]:
### create sport df: four

# function to return last word in city+team or team
def team_name(row):
    return (row[-1])

# main nfl2016df
nfldata = pd.read_csv('assets/nfl.csv')
# include only 2016
nfl2016mask=nfldata['year'] == 2016
nfl2016df = nfldata.where(nfl2016mask).dropna()
# clean NFL df
nfl2016df['team'] = nfl2016df['team'].str.replace('[*+]', '')
nfl2016df['team'] = nfl2016df['team'].str.split(" ")
nfl2016df['team'] = nfl2016df['team'].apply(team_name)

# main mlb2016df
mlbdata = pd.read_csv('assets/mlb.csv')
# include only 2016
mlb2016mask=mlbdata['year'] == 2016
mlb2016df = mlbdata.where(mlb2016mask).dropna()
# clean MLB df
mlb2016df['team'] = mlb2016df['team'].str.split(" ")
mlb2016df['team'] = mlb2016df['team'].apply(team_name)
# Manual code fixing of Red Sox, White Sox from Sox, Sox to Red, White
mlb2016df.at[60, 'team'] = 'Red'
mlb2016df.at[68, 'team'] = 'White'

# main nba2016df
nbadata = pd.read_csv('assets/nba.csv')
# include only 2016
nba2016mask=nbadata['year'] == 2016
nba2016df = nbadata.where(nba2016mask).dropna()
# clean NBA df
nba2016df['team'] = nba2016df['team'].str.replace('[*()0-9]', '')
nba2016df['team'] = nba2016df['team'].str.split(" ")
nba2016df['team'] = nba2016df['team'].apply(team_name)
### Manual code fixing of 'ers'
nba2016df.at[74, 'team'] = '76ers'

### main nhl2017df
nhldata = pd.read_csv('assets/nhl.csv')[1:]
#include only 2016: 
nhl2017df = nhldata[69:102] #2nd set of Atlant, Metro, Central, Pacific Division listed for #NHL
#clean NHL df
nhl2017df['team'] = nhl2017df['team'].str.replace('[\*]', '')
nhl2017df['team'] = nhl2017df['team'].str.split(" ")
nhl2017df['team'] = nhl2017df['team'].apply(team_name)

In [4]:
### split team names in wikidata

# NFL team names split
wikidata2 = wikidata.NFL.str.split('(?=[A-Z][a-z])').apply(pd.Series)
wikidata2.index = wikidata.set_index(['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']).index
wikidata2.stack().reset_index()
wikidata2.reset_index(inplace=True)

# MLB team names split
wikidata3 = wikidata.MLB.str.split('(?=[A-Z][a-z])').apply(pd.Series)
wikidata3.index = wikidata.set_index(['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']).index
wikidata3.stack().reset_index()
wikidata3.reset_index(inplace=True)

# NBA team names split
wikidata4 = wikidata.NBA.str.split('(?=[A-Z][a-z])').apply(pd.Series)
wikidata4.index = wikidata.set_index(['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']).index
wikidata4.stack().reset_index()
wikidata4.reset_index(inplace=True)

# NHL team names split
wikidata5 = wikidata.NHL.str.split('(?=[A-Z][a-z])').apply(pd.Series)
wikidata5.index = wikidata.set_index(['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']).index
wikidata5.stack().reset_index()
wikidata5.reset_index(inplace=True)

In [5]:
### merge wikidata frames with split sports names with sports df

# define function to calculate W-L ratio (W/(L+W))
#  this specific method was chosen since we are attempting to compare performance of teams
#  dividing by L+W makes the ratio dependent on all outcomes, and is seemingly common in league statistics
def add_ratio(row):
    return (float(row['W']))/((float(row['L']))+ float(row['W']))

# NFL merges, concatenation and application of add_ratio function to create ration column
wikidata2.rename(columns={0: "team"}, inplace=True)
merge1 = pd.merge(wikidata2, nfl2016df, on='team', how='inner')

wikidata2.rename(columns={"team": 0}, inplace=True)
wikidata2.rename(columns={1: "team"}, inplace=True)
merge2 = pd.merge(wikidata2, nfl2016df, on='team', how='inner')

wikidata2.rename(columns={"team": 1}, inplace=True)
wikidata2.rename(columns={2: "team"}, inplace=True)
merge3 = pd.merge(wikidata2, nfl2016df, on='team', how='inner')

nflfinaldf = pd.concat([merge1, merge2, merge3]).reset_index()

nflfinaldf['ratio'] = nflfinaldf.apply(add_ratio, axis=1)

# MLB merges, concatenation and application of add_ratio function to create ration column
wikidata3.rename(columns={1: "team"}, inplace=True)
wikidata3['team'] = wikidata3['team'].str.strip()
merge4 = pd.merge(wikidata3, mlb2016df, on='team', how='inner')

wikidata3.rename(columns={"team": 1}, inplace=True)
wikidata3.rename(columns={2: "team"}, inplace=True)
wikidata3['team'] = wikidata3['team'].str.strip()
merge5 = pd.merge(wikidata3, mlb2016df, on='team', how='inner')

wikidata3.rename(columns={"team": 2}, inplace=True)
wikidata3.rename(columns={3: "team"}, inplace=True)
wikidata3['team'] = wikidata3['team'].str.strip()
merge6 = pd.merge(wikidata3, mlb2016df, on='team', how='inner')

mlbfinaldf = pd.concat([merge4, merge5, merge6]).reset_index()
mlbfinaldf['ratio'] = mlbfinaldf.apply(add_ratio, axis=1)

# NBA merges, concatenation and application of add_ratio function to create ration column
wikidata4.rename(columns={0: "team"}, inplace=True)
wikidata4['team'] = wikidata4['team'].str.strip()
merge7 = pd.merge(wikidata4, nba2016df, on='team', how='inner')

wikidata4.rename(columns={'team': '0'}, inplace=True)
wikidata4.rename(columns={1: "team"}, inplace=True)
wikidata4['team'] = wikidata4['team'].str.strip()
nba2016df['team'] = nba2016df['team'].str.strip()
merge8 = pd.merge(wikidata4, nba2016df, on='team', how='inner')

wikidata4.rename(columns={"team": 1}, inplace=True)
wikidata4.rename(columns={2: "team"}, inplace=True)
wikidata4['team'] = wikidata4['team'].str.strip()
merge9 = pd.merge(wikidata4, nba2016df, on='team', how='inner')
merge9

nbafinaldf = pd.concat([merge7, merge8, merge9]).reset_index()
nbafinaldf['ratio'] = nbafinaldf.apply(add_ratio, axis=1)

# NHL merges, concatenation and application of add_ratio function to create ration column
wikidata5.rename(columns={1: "team"}, inplace=True)
wikidata5['team'] = wikidata5['team'].str.strip()
mergeA = pd.merge(wikidata5, nhl2017df, on='team', how='inner')

wikidata5.rename(columns={'team': '1'}, inplace=True)
wikidata5.rename(columns={2: "team"}, inplace=True)
wikidata5['team'] = wikidata5['team'].str.strip()
#nba2016df['team'] = nba2016df['team'].str.strip()
mergeB = pd.merge(wikidata5, nhl2017df, on='team', how='inner')

wikidata5.rename(columns={"team": 2}, inplace=True)
wikidata5.rename(columns={3: "team"}, inplace=True)
#wikidata5['team'] = wikidata4['team'].str.strip()
mergeC = pd.merge(wikidata5, nhl2017df, on='team', how='inner')

nhlfinaldf = pd.concat([mergeA, mergeB, mergeC]).reset_index()
nhlfinaldf['ratio'] = nhlfinaldf.apply(add_ratio, axis=1)

In [6]:
### prepare ordered lists of ratios to send into corr function

# definition of function to change population column to numeric
def pop_to_numeric(row):
    return (float(row['Population (2016 est.)[8]']))

# create NFL ordered list of ratios for correlation analysis
nflfinaldf = nflfinaldf.sort_values(by='Metropolitan area')
nflfinaldf['Population (2016 est.)[8]'] = nflfinaldf.apply(pop_to_numeric, axis=1)

nfl_pop_list = nflfinaldf.groupby(['Metropolitan area'])['Population (2016 est.)[8]'].mean().tolist()
nfl_ratio_list = nflfinaldf.groupby(['Metropolitan area'])['ratio'].mean().tolist()

# create MLB ordered list of ratios for correlation analys
mlbfinaldf = mlbfinaldf.sort_values(by='Metropolitan area')
mlbfinaldf['Population (2016 est.)[8]'] = mlbfinaldf.apply(pop_to_numeric, axis=1)

mlb_pop_list = mlbfinaldf.groupby(['Metropolitan area'])['Population (2016 est.)[8]'].mean().tolist()
mlb_ratio_list = mlbfinaldf.groupby(['Metropolitan area'])['ratio'].mean().tolist()

# create NBA ordered list of ratios for correlation analys
nbafinaldf = nbafinaldf.sort_values(by='Metropolitan area')
nbafinaldf['Population (2016 est.)[8]'] = nbafinaldf.apply(pop_to_numeric, axis=1)

nba_pop_list = nbafinaldf.groupby(['Metropolitan area'])['Population (2016 est.)[8]'].mean().tolist()
nba_ratio_list = nbafinaldf.groupby(['Metropolitan area'])['ratio'].mean().tolist()

# create NBA ordered list of ratios for correlation analys
nhlfinaldf = nhlfinaldf.sort_values(by='Metropolitan area')
nhlfinaldf['Population (2016 est.)[8]'] = nhlfinaldf.apply(pop_to_numeric, axis=1)

nhl_pop_list = nhlfinaldf.groupby(['Metropolitan area'])['Population (2016 est.)[8]'].mean().tolist()
nhl_ratio_list = nhlfinaldf.groupby(['Metropolitan area'])['ratio'].mean().tolist()

In [7]:
### calculate correlation between metro area and W-L ratios
nflcorr = nfl_corr, nfl_pval=stats.pearsonr(nfl_pop_list, nfl_ratio_list)
mlbcorr = mlb_corr, mlb_pval=stats.pearsonr(mlb_pop_list, mlb_ratio_list)
nbacorr = nba_corr, nba_pval=stats.pearsonr(nba_pop_list, nba_ratio_list)
nhlcorr = nhl_corr, nhl_pval=stats.pearsonr(nhl_pop_list, nhl_ratio_list)

In [8]:
### display correlation between metro area population and win-loss ratios
def nfl_correlation():
    return nflcorr
def mlb_correlation():
    return mlbcorr
def nba_correlation():
    return nbacorr
def nhl_correlation():
    return nhlcorr

print("the win/loss ratio's correlation with the population of the city it is in")
print('Sport: (Pearson’s correlation coefficient, two-tailed p-value)')
print('NFL:', nfl_correlation())
print('MLB:', mlb_correlation())
print('NBA:', nba_correlation())
print('NHL:', nhl_correlation())

the win/loss ratio's correlation with the population of the city it is in
Sport: (Pearson’s correlation coefficient, two-tailed p-value)
NFL: (-0.06221175680784659, 0.7485176122800969)
MLB: (0.24554628030517028, 0.22662980834926003)
NBA: (-0.18730545966425094, 0.3398695676323587)
NHL: (0.3981017251172357, 0.039724141141359456)


In [9]:
### create empty dataframe to hold ttest results
q5df = pd.DataFrame(0, columns = ['NFL' , 'MLB', 'NBA' , 'NHL'], index=['NFL', 'MLB', 'NBA' , 'NHL'])
q5df.index.name = 'sports'
q5df = q5df.replace(0,np.NaN)

In [10]:
### create dataframes to filter sports against each other (team exists in both sports in both metro areas)

# NFL_MLB
mlb_metros = mlbfinaldf['Metropolitan area'].tolist()
nfl_mask_mlb = nflfinaldf['Metropolitan area'].isin(mlb_metros)
nfl_mlb_df = nflfinaldf.where(nfl_mask_mlb).dropna(subset=['Metropolitan area'])

nfl_metros = nflfinaldf['Metropolitan area'].tolist()
mlb_mask_nfl = mlbfinaldf['Metropolitan area'].isin(nfl_metros)
mlb_nfl_df = mlbfinaldf.where(mlb_mask_nfl).dropna(subset=['Metropolitan area'])

#NFL_NBA
nba_metros = nbafinaldf['Metropolitan area'].tolist()
nfl_mask_nba = nflfinaldf['Metropolitan area'].isin(nba_metros)
nfl_nba_df = nflfinaldf.where(nfl_mask_nba).dropna(subset=['Metropolitan area'])

nfl_metros = nflfinaldf['Metropolitan area'].tolist()
nba_mask_nfl = nbafinaldf['Metropolitan area'].isin(nfl_metros)
nba_nfl_df = nbafinaldf.where(nba_mask_nfl).dropna(subset=['Metropolitan area'])

#NFL_NHL
nhl_metros = nhlfinaldf['Metropolitan area'].tolist()
nfl_mask_nhl = nflfinaldf['Metropolitan area'].isin(nhl_metros)
nfl_nhl_df = nflfinaldf.where(nfl_mask_nhl).dropna(subset=['Metropolitan area'])

nfl_metros = nflfinaldf['Metropolitan area'].tolist()
nhl_mask_nfl = nhlfinaldf['Metropolitan area'].isin(nfl_metros)
nhl_nfl_df = nhlfinaldf.where(nhl_mask_nfl).dropna(subset=['Metropolitan area'])

#MLB_NBA
nba_metros = nbafinaldf['Metropolitan area'].tolist()
mlb_mask_nba = mlbfinaldf['Metropolitan area'].isin(nba_metros)
mlb_nba_df = mlbfinaldf.where(mlb_mask_nba).dropna(subset=['Metropolitan area'])

mlb_metros = mlbfinaldf['Metropolitan area'].tolist()
nba_mask_mlb = nbafinaldf['Metropolitan area'].isin(mlb_metros)
nba_mlb_df = nbafinaldf.where(nba_mask_mlb).dropna(subset=['Metropolitan area'])

#MLB_NHL
nhl_metros = nhlfinaldf['Metropolitan area'].tolist()
mlb_mask_nhl = mlbfinaldf['Metropolitan area'].isin(nhl_metros)
mlb_nhl_df = mlbfinaldf.where(mlb_mask_nhl).dropna(subset=['Metropolitan area'])

mlb_metros = mlbfinaldf['Metropolitan area'].tolist()
nhl_mask_mlb = nhlfinaldf['Metropolitan area'].isin(mlb_metros)
nhl_mlb_df = nhlfinaldf.where(nhl_mask_mlb).dropna(subset=['Metropolitan area'])

#NHL_NBA
nba_metros = nbafinaldf['Metropolitan area'].tolist()
nhl_mask_nba = nhlfinaldf['Metropolitan area'].isin(nba_metros)
nhl_nba_df = nhlfinaldf.where(nhl_mask_nba).dropna(subset=['Metropolitan area'])

nhl_metros = nhlfinaldf['Metropolitan area'].tolist()
nba_mask_nhl = nbafinaldf['Metropolitan area'].isin(nhl_metros)
nba_nhl_df = nbafinaldf.where(nba_mask_nhl).dropna(subset=['Metropolitan area'])

In [11]:
### prepare ordered lists of win-loss ratios for ttest analysis

nfl_mlb_df = nfl_mlb_df.sort_values(by='Metropolitan area')
mlb_nfl_df = mlb_nfl_df.sort_values(by='Metropolitan area')
nfl_mlb_list = nfl_mlb_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
mlb_nfl_list = mlb_nfl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

nfl_nba_df = nfl_nba_df.sort_values(by='Metropolitan area')
nba_nfl_df = nba_nfl_df.sort_values(by='Metropolitan area')
nfl_nba_list = nfl_nba_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
nba_nfl_list = nba_nfl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

nfl_nhl_df = nfl_nhl_df.sort_values(by='Metropolitan area')
nhl_nfl_df = nhl_nfl_df.sort_values(by='Metropolitan area')
nfl_nhl_list = nfl_nhl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
nhl_nfl_list = nhl_nfl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

mlb_nba_df = mlb_nba_df.sort_values(by='Metropolitan area')
nba_mlb_df = nba_mlb_df.sort_values(by='Metropolitan area')
mlb_nba_list = mlb_nba_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
nba_mlb_list = nba_mlb_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

mlb_nhl_df = mlb_nhl_df.sort_values(by='Metropolitan area')
nhl_mlb_df = nhl_mlb_df.sort_values(by='Metropolitan area')
mlb_nhl_list = mlb_nhl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
nhl_mlb_list = nhl_mlb_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

nhl_nba_df = nhl_nba_df.sort_values(by='Metropolitan area')
nba_nhl_df = nba_nhl_df.sort_values(by='Metropolitan area')
nhl_nba_list = nhl_nba_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()
nba_nhl_list = nba_nhl_df.groupby(['Metropolitan area'])['ratio'].mean().tolist()

In [12]:
### perform ttest analyses, populate empty dataframe, display p-values in dataframe

x_nfl_mlb, p_nfl_mlb = ttest_rel(nfl_mlb_list, mlb_nfl_list)
q5df.at['NFL', 'MLB'] = p_nfl_mlb
q5df.at['MLB', 'NFL'] = p_nfl_mlb

x_nfl_nba, p_nfl_nba = ttest_rel(nfl_nba_list, nba_nfl_list)
q5df.at['NFL', 'NBA'] = p_nfl_nba
q5df.at['NBA', 'NFL'] = p_nfl_nba

x_nfl_nhl, p_nfl_nhl = ttest_rel(nfl_nhl_list, nhl_nfl_list)
q5df.at['NFL', 'NHL'] = p_nfl_nhl
q5df.at['NHL', 'NFL'] = p_nfl_nhl

x_mlb_nba, p_mlb_nba = ttest_rel(mlb_nba_list, nba_mlb_list)
q5df.at['MLB', 'NBA'] = p_mlb_nba
q5df.at['NBA', 'MLB'] = p_mlb_nba

x_mlb_nhl, p_mlb_nhl = ttest_rel(mlb_nhl_list, nhl_mlb_list)
q5df.at['MLB', 'NHL'] = p_mlb_nhl
q5df.at['NHL', 'MLB'] = p_mlb_nhl

x_nhl_nba, p_nhl_nba = ttest_rel(nhl_nba_list, nba_nhl_list)
q5df.at['NHL', 'NBA'] = p_nhl_nba
q5df.at['NBA', 'NHL'] = p_nhl_nba

def sports_team_performance():
    return q5df
sports_team_performance()

,NFL,MLB,NBA,NHL
sports,,,,
NFL,NaN,0.474534,0.933120,0.135340
MLB,0.474534,NaN,0.801798,0.000209
NBA,0.933120,0.801798,NaN,0.066390
NHL,0.135340,0.000209,0.066390,NaN


# Combined answers here:

Based on an alpha of 0.05, we can reject the null hypothesis that there is no difference between sport team performance for the following combination of sports teams within a metropolitan area:
MLB and NHL
For all other combinations of sports teams within a metropolitan area, we are unable to reject the null hypothesis that there is no difference in performance between the two types of teams since the p-values are above alpha.

In [13]:
print("the win/loss ratio's correlation with the population of the city it is in")
print('Sport: (Pearson’s correlation coefficient, two-tailed p-value)')
print('NFL:', nfl_correlation())
print('MLB:', mlb_correlation())
print('NBA:', nba_correlation())
print('NHL:', nhl_correlation())

sports_team_performance()

the win/loss ratio's correlation with the population of the city it is in
Sport: (Pearson’s correlation coefficient, two-tailed p-value)
NFL: (-0.06221175680784659, 0.7485176122800969)
MLB: (0.24554628030517028, 0.22662980834926003)
NBA: (-0.18730545966425094, 0.3398695676323587)
NHL: (0.3981017251172357, 0.039724141141359456)


,NFL,MLB,NBA,NHL
sports,,,,
NFL,NaN,0.474534,0.933120,0.135340
MLB,0.474534,NaN,0.801798,0.000209
NBA,0.933120,0.801798,NaN,0.066390
NHL,0.135340,0.000209,0.066390,NaN
